# Signals in the pump-probe geometry
Trying pp, 2D and 2D with scatter experiments

In [41]:
from __future__ import division, print_function
#from sympy import latex
import sympy as sy
from math import copysign
from operator import add, mul
import numpy as np
from functools import partial

In [2]:
from sig2d.signals import Signal, amplitude, signal, signals_for_order

In [3]:
sy.init_printing()

In [42]:
# utilites and filters
def fundamental_freq(k):
    return abs(sum([copysign(1,i) for i in k])) == 1

#def positive_last(k):
#    return k[-1] > 0

def multifilter(filters, reductop=all):
    def composite(k):
        return reductop([f(k) for f in filters])
    return composite

def accumulator(l, op=add):
    """
    Returns the running accumulation of `l` using operator `op`.
    
    accumulate(l) results in the running sum.
    """
    l = iter(l)
    r = l.next()
    for v in l:
        r = op(r, v)
        yield r
        
def accumulated(l, op=add):
    """Returns an running accumulation of `l` as a list."""
    return list(accumulator(l, op))
        

#def ends_with_k3(k):
#    return abs(k[-1]) == 3

#def single_k3(k):  # emitted in the k3 direction?
#    return sum([abs(i)==3 for i in k]) == 1

# emit in k3 direction
def emit_k3(k):
    # total +1 in k3 direction
    # others cancel out
    return (sum([copysign(1, i) for i in k if abs(i)==3]) == 1) and (sum([copysign(1, i) for i in k if abs(i)!=3]) == 0)

#def rwa(k):  # avoid +k1 +k1 sequences, which yeild |2><0| or try to deexcite the gs
#    # never get more than +1 or -1 total excitation, is that allright? # actually wrong. we can have |2><0| in two steps
#    runsum = accumulated([copysign(1, i) for i in k]) 
#    return all([abs(r) <= 1 for r in runsum])

def contains(s, sig):
    """Signal contains string s"""
    return s in repr(sig)

def contains_filter(s):
    return partial(contains, s)

#pp_2d = multifilter([fundamental_freq, positive_last, single_k3, single_excitation])
filter_pp = multifilter([fundamental_freq, emit_k3])

## clean 2d pp, 2 step

In [76]:
sigs = sum(signals_for_order(i, n_pulses=3, filter_=filter_pp) for i in [1,2,3])
sigs

In [77]:
amps = {i: amplitude(i) for i in [1,2,3]}
probe_field = amps[3]*sy.symbols("E_3")
probe_field
detected = sigs + probe_field
detected = sy.expand(sy.conjugate(detected)*detected)
# prepare further neglect
head = detected.func
args = detected.args
args = filter(contains_filter("E_3"), args)
detected = head(*args)

In [78]:
# phase cycling
phases = [0, sy.pi]
# Next line follows Tan's nomenclature
factors = [sy.exp(sy.I*p) for p in phases]
sig = sum([f*detected.subs([(amps[1], 1),
                            (amps[3], 1),
                            (amps[2], sy.exp(-sy.I*p))])
           for f, p in zip(factors, phases)]
    )
sy.re(sig.expand(complex=True))
sig

## 2d pp, with scatter

In [49]:
sigs = sum(signals_for_order(i, n_pulses=3, filter_=filter_pp) for i in [1,2,3])
sigs

In [60]:
amps = {i: amplitude(i) for i in [1,2,3]}
probe_field = amps[3]*sy.symbols("E_3")
alpha = sy.symbols(r'\alpha', real=True)
scatter = alpha*(amps[1]*sy.symbols("E_1")+amps[2]*sy.symbols("E_2"))
scatter

In [61]:

detected = sigs + probe_field + scatter
detected = sy.expand(sy.conjugate(detected)*detected)
# prepare further neglect
head = detected.func
args = detected.args
# remove terms depending on double responses (vanishingly small)
args = filter(lambda s: repr(s).count(r'\chi_')<2, args)
# remove scatter-response cross-terms (small) ... maybe that is not needed
args = filter(lambda s: not (r'\chi' in repr(s) and r'\alpha'  in repr(s)), args)
detected = head(*args)
detected

### 2-step

In [63]:
phases = [0, sy.pi]
# Next line follows Tan's nomenclature
factors = [sy.exp(sy.I*p) for p in phases]
sig = sum([f*detected.subs([(amps[1], 1),
                            (amps[3], 1),
                            (amps[2], sy.exp(-sy.I*p))])
           for f, p in zip(factors, phases)]
    )
sy.re(sig.expand(complex=True))
sig

### ogilvie 4 step scatter removal

In [72]:
#p1 = [sy.exp(-sy.I*i) for i in [0, 0, sy.pi, sy.pi]]
#p2 = [sy.exp(-sy.I*i) for i in [0, sy.pi, 0, sy.pi]]
phi1 = [0, 0, sy.pi, sy.pi]
phi2 = [0, sy.pi, sy.pi, 0]
factors = [1, -1, 1, -1]
sig = sum([f*detected.subs([(amps[1], sy.exp(-sy.I*p1)),
                            (amps[3], 1),
                            (amps[2], sy.exp(-sy.I*p2))])
           for f, p1, p2 in zip(factors, phi1, phi2)]
    )
sy.re(sig.expand(complex=True))
sig

### ogilvie 8 step scatter removal (cycling + chopping)

In [74]:
phi1 = [0, 0, 0, 0, sy.pi, sy.pi, sy.pi, sy.pi]
phi2 = [0, 0, sy.pi, sy.pi,sy.pi, sy.pi, 0, 0]
amp3 = [1, 0, 1, 0, 1, 0, 1, 0]
factors = [1, -1, -1, 1, 1, -1, -1, 1]
sig = sum([f*detected.subs([(amps[1], sy.exp(-sy.I*p1)),
                            (amps[3], a3),
                            (amps[2], sy.exp(-sy.I*p2))])
           for f, p1, p2, a3 in zip(factors, phi1, phi2, amp3)]
    )
sy.re(sig.expand(complex=True))
sig